In [ ]:
"""
Mark Jay 코드 기반으로 함.
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.fftpack import ifft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
RATE =  192000              # samples per second , 96000Hz
FRE_RESOLUTION = 10            # frequency domain resolution 
CHUNK =  RATE // FRE_RESOLUTION      # samples per frame,   // 는 소수점을 버리고 정수만 취한다. 
FORMAT = pyaudio.paInt16         # audio format (bytes per sample?)
CHANNELS = 1                     # single channel for microphone
SECOND_PER_FRAME = CHUNK / RATE  # second per frame, 한 프레임당 걸리는 시간 
BLOCKING_LOW_FREQUENCY = 100     # LOW range frequency blocking filter, Hz
BLOCKING_HIGH_FREQUENCY = 10000  # HIGH range frequency blocking filter, Hz
SHIFT_FREQUENCY = 30000         # 주파수 data를 이동시킬 주파수 


In [ ]:
print(CHUNK, 1/SECOND_PER_FRAME, BLOCKING_LOW_FREQUENCY // FRE_RESOLUTION, BLOCKING_HIGH_FREQUENCY // FRE_RESOLUTION, SHIFT_FREQUENCY//FRE_RESOLUTION )

In [ ]:

t = np.array(np.linspace(0, SECOND_PER_FRAME, CHUNK)) # len(t) = CHUNK

frequency = np.array(np.arange(0, RATE, 1/SECOND_PER_FRAME ))  # len(frequencty) = CHUNK, 0 부터 RATE까지 1/SECOND_PER_FRAME 간격으로 만든 행렬

# 1/SECOND_PER_FRAME, len(t), len(frequency)

In [ ]:
# create matplotlib figure and axes
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(10, 5))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer= CHUNK
)

# # create a line object with random data 실시간 plot을 위한 line 함수들 
line, = ax1.plot(t, np.random.rand(CHUNK), '-', lw=2)  # CHUNK갯수 만큼의 rand(0~1 사이의 랜덤 숫자 생성) 행렬을 만든다. 
line_fft, = ax2.plot(frequency, np.random.rand(CHUNK), '.', lw=2)  # input_sound_fft용
line_ifft, = ax3.plot(t, np.random.rand(CHUNK), '--', lw=2)          # soundagain ifft용


# format waveform axes
ax1.set_ylim(0, 255)
ax1.set_xlim(0, SECOND_PER_FRAME)

ax3.set_xlim(0, SECOND_PER_FRAME)
ax3.set_ylim(0, 255)          # ifft ax3 의 ylim 범위 
# format spectrum axes
ax2.set_xlim(20, RATE)
ax2.set_ylim(-0.1, 1.3)


# for measuring frame rate
print('stream started')
frame_count = 0
start_time = time.time()
    
while True:

    # binary data   
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    # data_np = np.array(data_int, dtype='b')[::2] + 128 

    data_np = np.array(data_int, dtype='b')[::2] + 128  

    line.set_ydata(data_np) # line(ax1)dp y데이터 입력 

    # compute FFT and update line
    yf = fft(data_int)  # yf is complex data and length is 2 x CHUNK 
    # yfft = np.abs(yf[0:])   # length = 2 * CHUNK
    yfft = np.abs(yf[0:CHUNK]) # length =  CHUNK  이것이 좋다. RATE을 넘어가는 hz는 의미가 없다. 
    
    # FFT data manipulation - cut the low and high frequency 
    yfft[0:BLOCKING_LOW_FREQUENCY // FRE_RESOLUTION + 1] = 0   # 0hz~ BLOCKING_LOW_FREQUENCY Hz 까지 0으로 만든다. 
    yfft[BLOCKING_HIGH_FREQUENCY // FRE_RESOLUTION:] = 0    # BLOCKING_HIGH_FREQUENCY Hz ~ 끝까지 0으로 만든다. 

    # FFT data manipulation - shift total frequency data 
    yfft[(SHIFT_FREQUENCY // FRE_RESOLUTION) : (SHIFT_FREQUENCY // FRE_RESOLUTION + BLOCKING_HIGH_FREQUENCY // FRE_RESOLUTION)] = \
     yfft[0:BLOCKING_HIGH_FREQUENCY // FRE_RESOLUTION ]  #  0 ~ HIHG_FRE.까지의 fft 데이터를 SHIFT_FRE. ~ SHIFT_FRE.+HIHG_FRE.로 복사함. 

    # remove the original fft data(0 ~ BLOCKING_HIGH_FREQUENCY Area) 
    yfft[0:BLOCKING_HIGH_FREQUENCY // FRE_RESOLUTION ] = 0 


    # fft data update line
    line_fft.set_ydata(yfft / (128 * CHUNK))

    # compute IFFT and update line
    iyfft = (ifft(yfft) / (4.1) )  + 128
    # print(np.real(iyfft[2500:2505]))
    line_ifft.set_ydata(np.real(iyfft))


# line_fft.set_ydata(yf)
# line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))  # line_fft(ax2)에 y데이터 입력 

    # update figure canvas

    # fig.canvas.draw()
    # fig.canvas.flush_events()
# plt.plot(t, np.abs(soundagain))   # soundagin의 abs와 real 은 동일한 결과를 준다. 
# plt.show()
    
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break


In [9]:
a = np.arange(0, 5, 0.1)
a


array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9])